#### 1. Import pandas library

In [1]:
import pandas as pd

#### 2. Import users table:

In [2]:
users_tables = pd.read_csv("./users_table.csv")
users_tables.head(5)

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\r\n\r\n<p>...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,NaN


#### 3. Rename Id column to userId

In [3]:
users_tables=users_tables.rename(columns={'Id':'userId'
                            })
users_tables

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\r\n\r\n<p>...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40320,55743,1,2014-09-13 21:03:50,AussieMeg,2014-09-13 21:18:52,NaN,NaN,NaN,0,0,0,5026902,NaN,http://graph.facebook.com/665821703/picture?ty...
40321,55744,6,2014-09-13 21:39:30,Mia Maria,2014-09-13 21:39:30,NaN,NaN,NaN,1,0,0,5026998,NaN,NaN
40322,55745,101,2014-09-13 23:45:27,tronbabylove,2014-09-13 23:45:27,NaN,United States,NaN,0,0,0,481766,NaN,https://www.gravatar.com/avatar/faa7a3fdbd8308...
40323,55746,106,2014-09-14 00:29:41,GPP,2014-09-14 02:05:17,NaN,NaN,"<p>Stats noobie, product, marketing &amp; medi...",1,0,0,976289,NaN,https://www.gravatar.com/avatar/6d9e9fa6b783a3...


#### 4. Import posts table:

In [4]:
post_table = pd.read_csv("./posts_table.csv")

#### 5. Rename Id column to postId and OwnerUserId to userId

In [5]:
post_table=post_table.rename(columns={'Id':'postId', 'OwnerUserId' : 'userId'
                            })
post_table.sort_values("userId")

,postId,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,userId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
18435,20262,4,NaN,2011-12-27 05:02:52,0,NaN,Statistical classification is the problem of i...,-1.0,2011-12-27 08:23:37,NaN,...,NaN,0,NaN,NaN,2011-12-27 08:23:37,NaN,NaN,NaN,NaN,user7997
29349,32803,5,NaN,2012-07-23 00:53:35,0,NaN,NaN,-1.0,2012-07-23 00:53:35,NaN,...,NaN,0,NaN,-1.0,2012-07-23 00:53:35,NaN,NaN,NaN,NaN,NaN
25347,28240,5,NaN,2012-05-11 03:14:43,0,NaN,NaN,-1.0,2012-05-11 03:14:43,NaN,...,NaN,0,NaN,-1.0,2012-05-11 03:14:43,NaN,NaN,NaN,NaN,NaN
36222,43800,5,NaN,2012-11-16 23:52:26,0,NaN,NaN,-1.0,2012-11-16 23:52:26,NaN,...,NaN,0,NaN,-1.0,2012-11-16 23:52:26,NaN,NaN,NaN,NaN,NaN
25365,28260,5,NaN,2012-05-11 13:00:09,0,NaN,NaN,-1.0,2012-05-11 13:00:09,NaN,...,NaN,0,NaN,-1.0,2012-05-11 13:00:09,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38182,46117,1,NaN,2012-12-17 20:14:30,0,355.0,<p>I used a stepwise multiple regression to ge...,NaN,2012-12-24 03:20:34,Variance explained from factors for step-wise ...,...,2.0,0,NaN,NaN,NaN,NaN,NaN,NaN,mettle,NaN
38306,46260,1,NaN,2012-12-19 18:06:20,5,145.0,<p>I'm caring monte-carlo simulations and I am...,NaN,2012-12-20 02:33:38,"Where can I learn about transforming uniform, ...",...,2.0,5,NaN,NaN,NaN,NaN,NaN,NaN,Yotam,NaN
38789,46836,1,NaN,2012-12-31 10:22:45,3,406.0,<p>I have got a set of elements $X$ which I ca...,NaN,2013-07-02 00:10:31,Machine learning algorithm for ranking,...,3.0,0,1.0,930.0,2013-01-02 20:49:20,NaN,NaN,NaN,almathie,NaN
38837,46892,2,NaN,2013-01-03 15:35:44,1,NaN,"<p>You can <a href=""http://en.wikipedia.org/wi...",NaN,2013-01-03 15:35:44,NaN,...,NaN,0,NaN,NaN,NaN,NaN,46891.0,NaN,Hélio Santos,NaN


#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [6]:
users_tables.columns

Index(['userId', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'AccountId', 'Age', 'ProfileImageUrl'],
      dtype='object')

In [7]:
users = users_tables[["userId", "Reputation", "Views", "UpVotes","DownVotes"]]
posts = post_table[["postId", "Score","userId","ViewCount","CommentCount"]]




In [19]:
posts["userId"] = posts["userId"].fillna(0)

posts= posts.astype({"userId" : "int64"},errors="ignore" )
posts


,postId,Score,userId,ViewCount,CommentCount
0,1,23,8,1278.0,1
1,2,22,24,8198.0,1
2,3,54,18,3613.0,4
3,4,13,23,5224.0,2
4,5,81,23,NaN,3
...,...,...,...,...,...
39995,48321,0,19966,NaN,0
39996,48322,3,892,NaN,2
39997,48323,1,2020,NaN,0
39998,48324,3,19914,NaN,0


#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [20]:
users_posts = pd.merge(users, posts, on= "userId")
users_posts.head(10)

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,NaN,0
1,-1,1,0,5007,1920,8576,0,NaN,0
2,-1,1,0,5007,1920,8578,0,NaN,0
3,-1,1,0,5007,1920,8981,0,NaN,0
4,-1,1,0,5007,1920,8982,0,NaN,0
5,-1,1,0,5007,1920,9857,0,NaN,0
6,-1,1,0,5007,1920,9858,0,NaN,0
7,-1,1,0,5007,1920,9860,0,NaN,0
8,-1,1,0,5007,1920,10130,0,NaN,0
9,-1,1,0,5007,1920,10131,0,NaN,0


#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [21]:
countmissing = users_posts.isna().sum()
countmissing

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
ViewCount       23572
CommentCount        0
dtype: int64

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [22]:
#see if all values from ViewCount missing. if yes, clean them , if not fill them 
users_posts.shape
if users_posts["ViewCount"].isna().sum() == users_posts.shape[0]:
    print("clean")
else:
    print("fill")
#fill missing with values with 0 like this we can still sort or perform operations. 
#no real info to put it at the mean or another arithmetical value, or ffill-bfill
#here actually made me think about the merge, not sure it is correct the way i did it.

users_posts["ViewCount"]= users_posts["ViewCount"].fillna(0)


fill


In [23]:
countmissing = users_posts.isna().sum()
countmissing


userId          0
Reputation      0
Views           0
UpVotes         0
DownVotes       0
postId          0
Score           0
ViewCount       0
CommentCount    0
dtype: int64

#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [77]:
users_posts.head(200)


,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,0.0,0
1,-1,1,0,5007,1920,8576,0,0.0,0
2,-1,1,0,5007,1920,8578,0,0.0,0
3,-1,1,0,5007,1920,8981,0,0.0,0
4,-1,1,0,5007,1920,8982,0,0.0,0
...,...,...,...,...,...,...,...,...,...
195,5,6792,1145,662,5,2166,4,0.0,0
196,5,6792,1145,662,5,2167,11,709.0,3
197,5,6792,1145,662,5,2172,15,0.0,3
198,5,6792,1145,662,5,2213,15,3182.0,2
